In [1]:
import pandas as pd
import boto3
import numpy as np
import os
import warnings
from pathlib import Path
from progressbar import ProgressBar
warnings.filterwarnings("ignore")
#set path directory
cwd = os.getcwd()


In [2]:
#load access key
keypath = f"{cwd}/AWSaccessKeys.csv"
access = pd.read_csv(keypath)


#start session
session = boto3.Session(
    aws_access_key_id=access['Access key ID'][0],
    aws_secret_access_key=access['Secret access key'][0],
)
s3 = session.resource('s3')

In [4]:
#load data
alldata = 'Data_1980-2020'
#state = 'ar'
model = 'NWM-AWS'
states = ['az', 'ca', 'co', 'ct', 'fl', 'ga', 'ia', 'id', 'il', 'in', 'ks']

for state in states:
    print(state)
    filepath = f"{cwd}/{alldata}/{model}/{state}/"
    files = os.listdir(filepath)

    pbar = ProgressBar()    
    for file in pbar(files):
        s3.meta.client.upload_file(Filename= f"{filepath}/{file}", Bucket='streamflow-app-data', Key=f"NWM_v2.1/NHD_segments_{state}.h5/{file}")

   

az


100% |########################################################################|


ca


100% |########################################################################|


co


...

100% |########################################################################|


in


100% |########################################################################|


ks


100% |########################################################################|


In [20]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import os #this is for joining the download directory
import pandas as pd
import geopandas as gpd

#AWS bucket information
#bucket_name = 'streamflow-app-data'
#s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED)
#bucket = s3.Bucket(bucket_name)

h=2
HUCunit = 'WBDHU4'
HU ='16'
bucket_name = 'streamflow-app-data'
filepath = f"s3://{bucket_name}/WBD/WBD_{HU}_HU2_GDB/WBD_{HU}_HU2_GDB.gdb/"
HUC_G = gpd.read_file(filepath, layer=HUCunit, config=Config(signature_version=UNSIGNED))
HUC_G


,tnmid,metasourceid,sourcedatadesc,sourceoriginator,sourcefeatureid,loaddate,referencegnis_ids,areaacres,areasqkm,states,huc4,name,shape_Length,shape_Area,geometry
0,{34090FE5-0C90-4DEE-8F9B-5ED942C5F3C9},NaN,NaN,NaN,NaN,2017-02-01T02:51:25+00:00,NaN,18329432.95,74176.65,"CA,NV,OR",1604,Black Rock Desert-Humboldt,32.944769,7.932010,"MULTIPOLYGON (((-117.96927 42.34301, -117.9685..."
1,{DFCBD7CC-EEC1-45DB-BDE3-359B922E9E41},NaN,NaN,NaN,NaN,2020-11-30T10:59:15+00:00,NaN,30543216.09,123604.12,"CA,NV",1606,Central Nevada Desert Basins,46.175343,12.791758,"MULTIPOLYGON (((-114.72112 41.24108, -114.7207..."
2,{7492C028-E858-42FB-848C-A49CB81CEE4B},NaN,NaN,NaN,NaN,2020-11-30T11:08:19+00:00,NaN,4809334.25,19462.70,"ID,UT,WY",1601,Bear,12.234765,2.111619,"MULTIPOLYGON (((-111.64623 42.85187, -111.6458..."
3,{06B7EBE2-D2DA-479C-BC22-51C3A8C0B2BF},NaN,NaN,NaN,NaN,2020-11-23T16:34:27+00:00,NaN,10543992.85,42670.06,"NV,UT",1603,Escalante Desert-Sevier Lake,15.003747,4.415539,"MULTIPOLYGON (((-112.17768 40.01123, -112.1773..."
4,{0001863F-AAE0-4036-B670-787FC04B98C3},NaN,NaN,NaN,NaN,2020-11-25T13:57:35+00:00,NaN,8115068.65,32840.55,"CA,NV",1605,Central Lahontan,16.137344,3.429845,"MULTIPOLYGON (((-119.09368 40.65821, -119.0934..."
5,{98264C17-0E40-4139-9A8A-516BC4A9414F},NaN,NaN,NaN,NaN,2020-11-30T11:08:18+00:00,NaN,18358651.93,74294.89,"ID,NV,UT,WY",1602,Great Salt Lake,23.967863,7.895899,"MULTIPOLYGON (((-112.66223 42.47056, -112.6622..."


In [3]:
filepath

's3://streamflow-app-data/WBD/WBD_16_HU2_GDB/WBD_16_HU2_GDB.gdb/'

In [14]:
#AWS bucket information
bucket_name = 'streamflow-app-data'
s3 = boto3.resource('s3', region_name = 'us-east-1')
bucket = s3.Bucket(bucket_name)

#Load streamstats wiht lat long to get geolocational information
csv_key = 'Streamstats/Streamstats.csv'
obj = bucket.Object(csv_key)
body = obj.get()['Body']
NWIS_sites = pd.read_csv(body)
NWIS_sites

,Unnamed: 0,NWIS_site_id,NWIS_sitename,Drainage_area_mi2,Mean_Basin_Elev_ft,Perc_Forest,Perc_Develop,Perc_Imperv,Perc_Herbace,Perc_Slop_30,...,dec_long_va,alt_va,alt_acy_va,huc_cd,parm_cd,begin_date,end_date,drain_sqkm,state,state_id
0,0,2378780,"D'OLIVE CREEK NEAR BRIDGEHEAD, AL",5.210000,NaN,NaN,53.90,11.50,NaN,NaN,...,-87.889647,NaN,NaN,NaN,60.0,2017-01-04,2020-10-25,NaN,Alabama,AL
1,1,2339495,OSELIGEE CREEK NEAR LANETT AL,86.300000,NaN,NaN,4.49,0.64,NaN,NaN,...,-85.196331,NaN,NaN,3130002.0,60.0,2006-07-26,2022-11-27,222.8292,Alabama,AL
2,2,2342500,"UCHEE CREEK NEAR FORT MITCHELL, AL.",322.000000,NaN,NaN,8.68,1.42,NaN,NaN,...,-85.014931,201.76,0.01,3130003.0,60.0,1946-10-01,2022-11-27,830.5740,Alabama,AL
3,3,2369800,BLACKWATER RIVER NEAR BRADLEY AL,0.044400,NaN,NaN,10.20,0.78,NaN,NaN,...,-86.709957,121.87,0.10,3140104.0,60.0,1967-10-01,2022-11-27,227.5129,Alabama,AL
4,4,2377570,"STYX RIVER NEAR ELSANOR, AL.",191.000000,NaN,NaN,4.72,0.74,NaN,NaN,...,-87.547206,NaN,NaN,3140106.0,60.0,1987-10-01,2022-11-27,495.4464,Alabama,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4964,4964,3213500,"PANTHER CREEK NEAR PANTHER, WV",31.000000,1780.0,NaN,3.50,NaN,NaN,NaN,...,-81.871111,1049.00,20.00,5070201.0,10.0,1973-05-10,1975-10-21,80.3709,West Virginia,WV
4965,4965,3214500,"TUG FORK AT KERMIT, WV",0.000077,626.0,NaN,100.00,NaN,NaN,NaN,...,-82.408758,574.07,20.00,5070201.0,10.0,2011-06-09,2018-10-19,3309.0550,West Virginia,WV
4966,4966,3206600,"EAST FORK TWELVEPOLE CREEK NEAR DUNLOW, WV",37.900000,1114.0,NaN,2.60,NaN,NaN,NaN,...,-82.305972,717.50,0.11,5090102.0,10.0,1974-03-14,1976-03-11,99.5850,West Virginia,WV
4967,4967,3207061,"TWELVEPOLE CREEK AT LAVALETTE, WV",402.000000,958.0,NaN,4.10,NaN,NaN,NaN,...,-82.448861,532.00,0.10,5090102.0,60.0,2019-11-14,2022-11-27,NaN,West Virginia,WV
